In [20]:
import plotly
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets

In [21]:
jd = pd.read_csv('./data/biblio_data.csv', sep=";", header=0)
alt = pd.read_csv('./data/cleaned_altmetrics.csv', sep=";", header=0)

In [22]:
data = alt.groupby('journal_name').agg(
    max=pd.NamedAgg(column='altmetric_score', aggfunc=max),
    min=pd.NamedAgg(column='altmetric_score', aggfunc=min),
    mean=pd.NamedAgg(column='altmetric_score', aggfunc=np.mean),
    median=pd.NamedAgg(column='altmetric_score', aggfunc=np.median),
)
data.reset_index(level=0, inplace=True)


In [23]:

select = widgets.Dropdown(
    options=[('Max', 'max'), ('Min', 'min'), ('Mean', 'mean'), ('Median', 'median')],
    value='max',
    description='Type:',
    disabled=False,
)
select

In [29]:
fig = go.Figure(go.Bar(
            x=data[select.value],
            y=data["journal_name"],
            orientation='h'))

# add vertival line for average
avg = np.mean(data[select.value])
fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'paper', y0= 0, y1= 1,
      xref= 'x', x0= avg, x1= avg,
      line=dict(color="red")
    )
])

fig.update_layout(title='Altmetric Score per Journal',
                   xaxis_title='Altmetric Score',
                   yaxis_title='Journal Name')

fig.show()

In [6]:
journal_names = data["journal_name"].tolist()

journals = widgets.Dropdown(
    options=journal_names,
    value=journal_names[0],
    description='Type:',
    disabled=False,
)
journals

In [7]:
data = pd.pivot_table(alt, index="journal_name", values=['news', 'blog', 'policy', 'patent', 'twitter', 'peer_review', 'weibo', 'facebook', 'wikipedia', 'google','linkedIn', 'reddit', 'pinterest', 'f1000', 'qa', 'videos', 'syllabi', 'mendeley'], aggfunc=np.sum)
data.reset_index(level=0, inplace=True)

labels = data.columns[1:].tolist()
filters = widgets.SelectMultiple(
    options=labels,
    value=labels[0::3],
    rows=10,
    description='Fruits',
    disabled=False
)
filters

Widget Javascript not detected.  It may not be installed or enabled properly.


In [16]:
selected_filters = filters.value
selected_journal = journals.value
row = data.loc[data["journal_name"] == selected_journal]
d = row.T
d = d.drop(d.index[[0]])
d.reset_index(level=0, inplace=True)
d.columns = ['labels', 'values']
d.values
fig = go.Figure(data=[go.Pie(labels=[x for x in d.labels if x in selected_filters], values=[x[1] for x in d.values if x[0] in selected_filters])])
fig.show()